In [17]:
import pandas as pd
import numpy as np
import librosa
import IPython.display as ipd
import matplotlib.pyplot as plt
import librosa.display
%matplotlib inline
import os
import pickle
from PIL import Image
import pathlib
import csv
from sklearn.model_selection import train_test_split
from sklearn_pandas import DataFrameMapper
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score
import tensorflow.keras as keras

import warnings
warnings.filterwarnings('ignore')

In [18]:
songs_df = pd.read_csv('song_data.csv')

In [19]:
songs_df.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,happy_015.mp3,0.457246,0.022467,2313.413622,2626.782970,4124.601950,0.056585,-537.234192,91.823860,35.459835,...,-2.303244,-2.529682,-0.756528,-0.102221,-0.657489,-0.360597,-0.160689,-1.208762,-1.592072,happy
1,happy_001.mp3,0.274254,0.058960,717.333558,713.542579,1129.112190,0.026445,-389.157745,218.432053,32.675011,...,-7.250138,-12.127299,-7.517781,-2.840759,0.056820,4.290653,5.128882,2.677651,0.546200,happy
2,happy_029.mp3,0.393490,0.063921,516.544569,647.894549,770.933663,0.017521,-404.102905,206.850693,68.775932,...,-10.706476,-8.817365,-1.754093,4.350704,2.414201,-2.776739,-2.774598,-0.309426,-2.359100,happy
3,happy_217.mp3,0.360551,0.081740,637.271723,614.921151,1020.346393,0.027209,-369.361420,211.137619,57.493961,...,-5.958784,-11.036363,-7.450860,0.486627,2.838610,-0.458699,-1.893717,-0.376333,-0.483401,happy
4,happy_203.mp3,0.338282,0.110650,640.259406,636.496897,1122.076094,0.024941,-322.983856,259.454285,33.289051,...,-1.428184,-3.662958,-0.929638,1.900791,-0.808726,-3.739435,-3.212300,-2.063034,-1.726896,happy


In [20]:
genre_list = songs_df.iloc[:, -1].tolist()

In [21]:
X = songs_df.iloc[:, :-1].drop(['filename'], axis = 1)
y = genre_list
X.head()

,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,0.457246,0.022467,2313.413622,2626.782970,4124.601950,0.056585,-537.234192,91.823860,35.459835,8.019248,...,-0.754915,-2.303244,-2.529682,-0.756528,-0.102221,-0.657489,-0.360597,-0.160689,-1.208762,-1.592072
1,0.274254,0.058960,717.333558,713.542579,1129.112190,0.026445,-389.157745,218.432053,32.675011,-19.545815,...,-2.030162,-7.250138,-12.127299,-7.517781,-2.840759,0.056820,4.290653,5.128882,2.677651,0.546200
2,0.393490,0.063921,516.544569,647.894549,770.933663,0.017521,-404.102905,206.850693,68.775932,-2.267009,...,-10.267504,-10.706476,-8.817365,-1.754093,4.350704,2.414201,-2.776739,-2.774598,-0.309426,-2.359100
3,0.360551,0.081740,637.271723,614.921151,1020.346393,0.027209,-369.361420,211.137619,57.493961,-11.652444,...,-1.013260,-5.958784,-11.036363,-7.450860,0.486627,2.838610,-0.458699,-1.893717,-0.376333,-0.483401
4,0.338282,0.110650,640.259406,636.496897,1122.076094,0.024941,-322.983856,259.454285,33.289051,-27.118906,...,-1.669189,-1.428184,-3.662958,-0.929638,1.900791,-0.808726,-3.739435,-3.212300,-2.063034,-1.726896


In [22]:
mapper = DataFrameMapper([(['chroma_stft'], [StandardScaler()]),
(['rmse'], [StandardScaler()]),
(['spectral_centroid'], [StandardScaler()]),
(['spectral_bandwidth'], [StandardScaler()]),
(['rolloff'], [StandardScaler()]),
(['zero_crossing_rate'], [StandardScaler()]),
(['mfcc1'], [StandardScaler()]),
(['mfcc2'], [StandardScaler()]),
(['mfcc3'], [StandardScaler()]),
(['mfcc4'], [StandardScaler()]),
(['mfcc5'], [StandardScaler()]),
(['mfcc6'], [StandardScaler()]),
(['mfcc7'], [StandardScaler()]),
(['mfcc8'], [StandardScaler()]),
(['mfcc9'], [StandardScaler()]),
(['mfcc10'], [StandardScaler()]),
(['mfcc11'], [StandardScaler()]),
(['mfcc12'], [StandardScaler()]),
(['mfcc13'], [StandardScaler()]),
(['mfcc14'], [StandardScaler()]),
(['mfcc15'], [StandardScaler()]),
(['mfcc16'], [StandardScaler()]),
(['mfcc17'], [StandardScaler()]),
(['mfcc18'], [StandardScaler()]),
(['mfcc19'], [StandardScaler()]),
(['mfcc20'], [StandardScaler()])],input_df = True,df_out=True)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=42)

In [24]:
X_train = mapper.fit_transform(X_train)
X_test = mapper.transform(X_test)

In [25]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [26]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(penalty = 'l2', C = 1.0, solver='saga')
logreg.fit(X_train, y_train)
y_pred = (logreg.predict(X_test)).round()
logreg.score(X_test, y_test)

0.9381443298969072

In [27]:
pipe = make_pipeline(mapper, logreg)
pipe.fit(X_train, y_train)
pipe.predict(X_test).round()
pickle.dump(pipe, open('pipe.pkl', 'wb'))

In [28]:
y_pred

array([1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0])

----------------------------------------------------------------------------------------------------------------------

### Working with Keras model

In [10]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(10, activation='relu', kernel_initializer = 'he_normal', input_shape=(X_train.shape[1],)))
model.add(Dense(20, kernel_initializer = 'he_normal', activation='relu'))
model.add(Dense(15, kernel_initializer = 'he_normal', activation='relu'))
model.add(Dense(1, activation='sigmoid'))

Using TensorFlow backend.


In [11]:
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [12]:
history = model.fit(X_train, y_train, epochs=100, batch_size = 10, validation_split = 0.33)

Train on 262 samples, validate on 130 samples
Epoch 1/100
262/262 [==============================] - 0s 858us/step - loss: 0.9659 - accuracy: 0.5534 - val_loss: 0.8012 - val_accuracy: 0.5615
Epoch 2/100
262/262 [==============================] - 0s 202us/step - loss: 0.6575 - accuracy: 0.6718 - val_loss: 0.5961 - val_accuracy: 0.7077
Epoch 3/100
262/262 [==============================] - 0s 159us/step - loss: 0.5635 - accuracy: 0.7405 - val_loss: 0.5137 - val_accuracy: 0.7692
Epoch 4/100
262/262 [==============================] - 0s 146us/step - loss: 0.4902 - accuracy: 0.7748 - val_loss: 0.4769 - val_accuracy: 0.8000
Epoch 5/100
262/262 [==============================] - 0s 137us/step - loss: 0.4317 - accuracy: 0.8168 - val_loss: 0.4329 - val_accuracy: 0.8308
Epoch 6/100
262/262 [==============================] - 0s 136us/step - loss: 0.3801 - accuracy: 0.8588 - val_loss: 0.4043 - val_accuracy: 0.8538
Epoch 7/100
262/262 [==============================] - 0s 136us/step - loss: 0.3349 

Epoch 57/100
262/262 [==============================] - 0s 113us/step - loss: 0.0042 - accuracy: 1.0000 - val_loss: 0.4745 - val_accuracy: 0.9077
Epoch 58/100
262/262 [==============================] - 0s 112us/step - loss: 0.0039 - accuracy: 1.0000 - val_loss: 0.4777 - val_accuracy: 0.9077
Epoch 59/100
262/262 [==============================] - 0s 112us/step - loss: 0.0037 - accuracy: 1.0000 - val_loss: 0.4805 - val_accuracy: 0.9077
Epoch 60/100
262/262 [==============================] - 0s 115us/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 0.4851 - val_accuracy: 0.9077
Epoch 61/100
262/262 [==============================] - 0s 117us/step - loss: 0.0034 - accuracy: 1.0000 - val_loss: 0.4879 - val_accuracy: 0.9077
Epoch 62/100
262/262 [==============================] - 0s 114us/step - loss: 0.0032 - accuracy: 1.0000 - val_loss: 0.4917 - val_accuracy: 0.9077
Epoch 63/100
262/262 [==============================] - 0s 111us/step - loss: 0.0031 - accuracy: 1.0000 - val_loss: 0.4920 -

In [13]:
model.evaluate(X_test, y_test)

194/194 [==============================] - 0s 23us/step


[0.526478635315086, 0.9432989954948425]

In [14]:
y_pred = (model.predict(X_test)).round()

In [15]:
accuracy_score(y_pred, y_test)

0.9432989690721649

In [16]:
### experiment
import dill
pipe = make_pipeline(mapper, model)
pipe.fit(X_train, y_train)
pipe.predict(X_test).round()
dill.dump(pipe, open('pipe.pkl', 'wb'))


Epoch 1/1
392/392 [==============================] - 0s 36us/step - loss: 0.1894 - accuracy: 0.9745
